In [20]:
import numpy as np
import pydicom as dicom
import matplotlib.pyplot as plt
from ipywidgets import interact
import json
import scipy.interpolate
import pandas as pd

In [1]:
import sys
sys.path.insert(0, "../src")


In [84]:
import bdfileutils as bdf
import bd4cnn.bd4cnn_metrics
import bdfileutils

In [126]:
import importlib
importlib.reload(bd4cnn.bd4cnn_metrics)
importlib.reload(bdfileutils)

<module 'bdfileutils' from '../src\\bdfileutils.py'>

In [110]:
pids = [
    '1405662',
    '1405045',
    '1404100',
    '1403302',
    '1402847',
    '1402419',
    '1402186',
    '1401571',
    '1400518',
    '1399784',
    '1399105',
    '1398889',
    '1398806',
    '1398511',
    '1397534'
]

In [137]:
def append_metrics(idx, pid, df, data):
    PTV_AVG = None
    PTV_D98 = None
    PTV_V95 = None
    HEART_AVG = None
    LUNGS_L_AVG = None
    LUNGS_R_AVG = None
    PTV_ROI = None
    PTV_TYPE = None

    for r, m in data['metrics'].items():
        if 'blizna PTV' in r:
            PTV_AVG = m['avg']
            PTV_D98 = m['d98']
            PTV_V95 = m['v95']
            PTV_ROI = r
            PTV_TYPE = 'blizna'
        elif 'boost PTV' in r:
            PTV_AVG = m['avg']
            PTV_D98 = m['d98']
            PTV_V95 = m['v95']
            PTV_ROI = r
            PTV_TYPE = 'boost/piers'
            
        if 'pluco L' in r:
            LUNGS_L_AVG = m['avg']
        if 'pluco P' in r:
            LUNGS_R_AVG = m['avg']
        if 'serce' in r:
            HEART_AVG = m['avg']
    
    df.loc[idx] = [pid, data['target_dose'], PTV_ROI, PTV_TYPE, PTV_AVG, PTV_D98, PTV_V95, HEART_AVG, LUNGS_L_AVG, LUNGS_R_AVG]
         
def create_df():
    # return pd.DataFrame(columns=['PID', 'TARGET_DOSE', 'ROI', 'TYPE', 'PTV MIN', 'PTV AVG', 'PTV MAX'])
    return pd.DataFrame(columns=['PID', 'TARGET_DOSE', 'PTV ROI', 'TYPE', 'PTV AVG', 'PTV D98%', 'PTV V95%', 'HEART AVG', 'LUNGS L AVG', 'LUNGS P AVG' ])

In [118]:
args = lambda x:x
args.single = True


    #args.roi_subfolder = 'output_18'
    #args.doses_subfolder_file = "output_50_e_31+41\doses_array.nparray"
    #args.doses_subfolder_file = "output_18_batch_8_doucz\doses_array.nparray"
    #args.doses_dtype='f4'
    #args.doses_dtype='u1'


In [138]:
df50 = create_df()
for idx, pid in enumerate(pids):
    folder = f'E:\\sim4\\{pid}'
    args.root_folder = folder
    args.roi_subfolder = 'output_50'
    args.doses_subfolder_file = "output_50\doses_to_max.nparray"
    args.doses_dtype='f4'
    
    res = bd4cnn.bd4cnn_metrics.do_run(args)
    append_metrics(idx, pid, df50, res[folder])
            


2021-02-04 16:30:41,430 [INFO]: Reading ndarray data from file: E:\sim4\1405662\output_50\rois_marks_original.nparray
2021-02-04 16:30:42,174 [INFO]: Reading ndarray data from file: E:\sim4\1405662\output_50\doses_to_max.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [    262144]  0.00 |   8.30 |   0.48 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [    131072]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  48.77 |   7.25 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [         2] 40.47 |  48.77 |  44.99 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 16:30:43,695 [INFO]: Reading ndarray data from file: E:\sim4\1405045\output_50\rois_marks_original.nparray
2021-02-04 16:30:44,299 [INFO]: Reading ndarray data from file: E:\sim4\1405045\output_50\doses_to_max.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 54.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [ 536870912]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [ 268435456]  0.00 |  20.17 |   0.36 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  55.77 |   7.90 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                         _4500 [  33554432] 26.10 |  52.21 |  44.99 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 16:30:46,062 [INFO]: Reading ndarray data from file: E:\sim4\1404100\output_50\rois_marks_original.nparray
2021-02-04 16:30:46,697 [INFO]: Reading ndarray data from file: E:\sim4\1404100\output_50\doses_to_max.nparray


...

------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 56.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [   8388608]  0.00 |   6.25 |   0.42 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [   4194304]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  57.52 |   7.54 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                         _4500 [34359738368]  0.00 |  52.52 |  45.29 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
 

2021-02-04 16:31:11,699 [INFO]: Reading ndarray data from file: E:\sim4\1398511\output_50\rois_marks_original.nparray
2021-02-04 16:31:12,512 [INFO]: Reading ndarray data from file: E:\sim4\1398511\output_50\doses_to_max.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [  16777216]  0.00 |   5.17 |   0.42 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [   8388608]  0.00 |  18.62 |   1.61 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  47.59 |   6.28 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [         8] 42.41 |  47.59 |  45.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 16:31:14,770 [INFO]: Reading ndarray data from file: E:\sim4\1397534\output_50\rois_marks_original.nparray
2021-02-04 16:31:15,401 [INFO]: Reading ndarray data from file: E:\sim4\1397534\output_50\doses_to_max.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [       512]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [       256]  0.00 |  13.73 |   0.38 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  47.52 |   7.12 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [        64] 42.24 |  47.52 |  45.01 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

In [139]:

df50_e_31_41 = create_df()
for idx, pid in enumerate(pids):
    folder = f'E:\\sim4\\{pid}'
    args.root_folder = folder
    args.roi_subfolder = 'output_50'
    args.doses_subfolder_file = "output_50_e_31+41\doses_array.nparray"
    args.doses_dtype='u1'
    
    res = bd4cnn.bd4cnn_metrics.do_run(args)
    append_metrics(idx, pid, df50_e_31_41, res[folder])
            

        


2021-02-04 16:31:17,105 [INFO]: Reading ndarray data from file: E:\sim4\1405662\output_50\rois_marks_original.nparray
2021-02-04 16:31:17,711 [INFO]: Reading ndarray data from file: E:\sim4\1405662\output_50_e_31+41\doses_array.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [    262144]  0.00 |  24.90 |   2.14 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [    131072]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  51.88 |   6.39 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [         2] 36.32 |  49.81 |  44.64 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 16:31:19,002 [INFO]: Reading ndarray data from file: E:\sim4\1405045\output_50\rois_marks_original.nparray
2021-02-04 16:31:19,608 [INFO]: Reading ndarray data from file: E:\sim4\1405045\output_50_e_31+41\doses_array.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 54.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [ 536870912]  0.00 |  28.48 |   0.92 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [ 268435456]  0.00 |  28.48 |   0.92 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  60.51 |   7.10 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                         _4500 [  33554432]  0.00 |  60.51 |  43.45 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 16:31:21,154 [INFO]: Reading ndarray data from file: E:\sim4\1404100\output_50\rois_marks_original.nparray
2021-02-04 16:31:21,790 [INFO]: Reading ndarray data from file: E:\sim4\1404100\output_50_e_31+41\doses_array.nparray


...

------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 56.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [   8388608]  0.00 |  21.26 |   0.88 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [   4194304]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  61.27 |   7.11 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                         _4500 [34359738368]  0.00 |  61.27 |  45.93 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
 

2021-02-04 16:31:44,388 [INFO]: Reading ndarray data from file: E:\sim4\1398511\output_50\rois_marks_original.nparray
2021-02-04 16:31:45,189 [INFO]: Reading ndarray data from file: E:\sim4\1398511\output_50_e_31+41\doses_array.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [  16777216]  0.00 |  17.59 |   1.60 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [   8388608]  0.00 |  17.59 |   1.63 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  51.72 |   5.36 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [         8] 24.83 |  49.66 |  44.51 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 16:31:47,178 [INFO]: Reading ndarray data from file: E:\sim4\1397534\output_50\rois_marks_original.nparray
2021-02-04 16:31:47,808 [INFO]: Reading ndarray data from file: E:\sim4\1397534\output_50_e_31+41\doses_array.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [       512]  0.00 |  22.17 |   0.99 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [       256]  0.00 |  22.17 |   0.98 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  55.96 |   6.28 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [        64] 39.07 |  51.74 |  46.27 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

In [145]:
df_18_batch_8_doucz = create_df()
for idx, pid in enumerate(pids):
    folder = f'E:\\sim4\\{pid}'
    args.root_folder = folder
    args.roi_subfolder = 'output_18'
    args.doses_subfolder_file = "output_18_batch_8_doucz\doses_array.nparray"
    args.single = True
    args.doses_dtype='u1'
    
    res = bd4cnn.bd4cnn_metrics.do_run(args)
    append_metrics(idx, pid, df_18_batch_8_doucz, res[folder])
            

2021-02-04 16:33:36,437 [INFO]: Reading ndarray data from file: E:\sim4\1405662\output_18\rois_marks_original.nparray
2021-02-04 16:33:37,185 [INFO]: Reading ndarray data from file: E:\sim4\1405662\output_18_batch_8_doucz\doses_array.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [    262144]  0.00 |  14.41 |   0.47 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [    131072]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  49.00 |   6.12 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [         2] 37.47 |  46.12 |  44.75 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 16:33:38,477 [INFO]: Reading ndarray data from file: E:\sim4\1405045\output_18\rois_marks_original.nparray
2021-02-04 16:33:39,085 [INFO]: Reading ndarray data from file: E:\sim4\1405045\output_18_batch_8_doucz\doses_array.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 54.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [ 536870912]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [ 268435456]  0.00 |  16.48 |   0.49 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  56.03 |   7.20 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                         _4500 [  33554432]  0.00 |  56.03 |  44.73 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 16:33:40,628 [INFO]: Reading ndarray data from file: E:\sim4\1404100\output_18\rois_marks_original.nparray
2021-02-04 16:33:41,261 [INFO]: Reading ndarray data from file: E:\sim4\1404100\output_18_batch_8_doucz\doses_array.nparray


...

------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 56.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [   8388608]  0.00 |  10.42 |   0.36 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [   4194304]  0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  59.05 |   7.30 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                         _4500 [34359738368]  0.00 |  59.05 |  46.47 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
 

2021-02-04 16:34:03,837 [INFO]: Reading ndarray data from file: E:\sim4\1398511\output_18\rois_marks_original.nparray
2021-02-04 16:34:04,634 [INFO]: Reading ndarray data from file: E:\sim4\1398511\output_18_batch_8_doucz\doses_array.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [  16777216]  0.00 |  11.49 |   0.73 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [   8388608]  0.00 |  14.37 |   1.15 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  48.85 |   5.52 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [         8] 37.36 |  45.98 |  45.29 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

2021-02-04 16:34:06,606 [INFO]: Reading ndarray data from file: E:\sim4\1397534\output_18\rois_marks_original.nparray
2021-02-04 16:34:07,236 [INFO]: Reading ndarray data from file: E:\sim4\1397534\output_18_batch_8_doucz\doses_array.nparray


------------------------------------------------------------------------------------------------------------------------
   Prescribed dose: 45.0
------------------------------------------------------------------------------------------------------------------------
           ROI Name            [   Bit    ] min   |  max   |  avg   |  d99%  |  d98%  |  d95%  |  V95%  |  V98%  | V105% 
------------------------------------------------------------------------------------------------------------------------
                 CouchInterior [       512]  0.00 |   5.87 |   0.05 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                  CouchSurface [       256]  0.00 |  11.73 |   0.41 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
               Patient Outline [         1]  0.00 |  52.79 |   6.19 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
                        blizna [        64] 41.06 |  52.79 |  46.96 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00 |   0.00
  

In [164]:
df50['PTV AVG - Target'] = df50['PTV AVG'] - df50['TARGET_DOSE']
df50

,PID,TARGET_DOSE,PTV ROI,TYPE,PTV AVG,PTV D98%,PTV V95%,HEART AVG,LUNGS L AVG,LUNGS P AVG,PTV AVG - Target
0,1405662,45.0,blizna PTV,blizna,44.765133,41.520828,91.834027,6.171153,12.954685,3.473011,-0.234867
1,1405045,54.0,boost PTV-plan,boost/piers,53.867287,50.879796,97.806181,3.238630,2.924156,12.754877,-0.132713
2,1404100,50.0,blizna PTV,blizna,39.427170,0.000000,65.686182,5.174376,2.546268,16.875298,-10.572830
3,1403302,56.0,boost PTV,boost/piers,55.782528,52.721765,94.392885,2.355551,1.863323,10.383221,-0.217472
4,1402847,45.0,blizna PTV,blizna,44.537739,40.621038,90.070352,3.374379,2.428270,13.021449,-0.462261
5,1402419,45.0,blizna PTV,blizna,44.733456,40.859781,94.666410,5.064692,3.117200,14.624176,-0.266544
6,1402186,56.0,boost PTV-plan,boost/piers,55.722382,52.606703,97.958412,4.422745,7.124682,2.119505,-0.277618
7,1401571,45.0,blizna PTV,blizna,44.703739,40.402903,94.135630,6.619849,14.076074,3.529819,-0.296261
8,1400518,56.0,boost PTV-plan,boost/piers,55.796410,52.901914,96.334851,4.208757,7.941990,3.084899,-0.203590
9,1399784,55.0,boost PTV-plan,boost/piers,54.354881,46.816000,90.202232,2.696232,2.489690,8.217636,-0.645119


In [165]:
df50.describe()

,TARGET_DOSE,PTV AVG,PTV D98%,PTV V95%,HEART AVG,LUNGS L AVG,LUNGS P AVG,PTV AVG - Target
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,51.000000,49.999208,43.486236,92.147111,4.554850,7.047349,7.527412,-1.000792
std,5.291503,6.053424,13.178328,7.869500,1.537378,4.737729,5.306488,2.652602
min,45.000000,39.427170,0.000000,65.686182,2.355551,1.863323,2.024131,-10.572830
25%,45.000000,44.718597,40.709506,90.249861,3.306504,2.536786,3.311176,-0.472712
50%,54.000000,53.867287,46.816000,94.392885,4.422745,7.124682,3.529819,-0.266544
75%,56.000000,55.752455,51.930331,96.563642,5.648648,10.949471,12.726439,-0.210531
max,56.000000,55.824306,52.901914,97.958412,7.373706,14.076074,16.875298,-0.132713


In [157]:

df50_e_31_41['PTV AVG - Target'] = df50_e_31_41['PTV AVG'] - df50_e_31_41['TARGET_DOSE']
df50_e_31_41

In [158]:
df50_e_31_41.describe()

,TARGET_DOSE,PTV AVG,PTV D98%,PTV V95%,HEART AVG,LUNGS L AVG,LUNGS P AVG,PTV AVG - Targe,PTV AVG - Target
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,51.000000,47.830218,26.095384,69.204775,4.813346,7.723306,8.409256,-3.169782,-3.169782
std,5.291503,4.690135,20.341190,29.316945,0.937161,4.957801,5.746628,3.466572,3.466572
min,45.000000,40.042413,0.000000,6.129918,3.503328,2.592414,2.487715,-9.957587,-9.957587
25%,45.000000,44.296316,0.000000,61.047512,3.904889,2.866528,3.349880,-4.807352,-4.807352
50%,54.000000,46.674739,32.024333,78.962682,4.989856,9.134956,4.310864,-1.847429,-1.847429
75%,56.000000,51.807024,44.405878,91.899217,5.438590,12.089558,13.674530,-0.901223,-0.901223
max,56.000000,55.148909,49.500191,94.218150,6.466756,15.097636,16.596582,0.751770,0.751770


In [160]:

df_18_batch_8_doucz['PTV AVG - Target'] = df_18_batch_8_doucz['PTV AVG'] - df_18_batch_8_doucz['TARGET_DOSE']
df_18_batch_8_doucz

,PID,TARGET_DOSE,PTV ROI,TYPE,PTV AVG,PTV D98%,PTV V95%,HEART AVG,LUNGS L AVG,LUNGS P AVG,PTV AVG - Target
0,1405662,45.0,blizna PTV,blizna,43.911255,37.071451,89.982964,4.911894,11.444135,3.182774,-1.088745
1,1405045,54.0,boost PTV-plan,boost/piers,54.817229,34.791518,94.524990,3.071679,2.626483,15.411094,0.817229
2,1404100,50.0,blizna PTV,blizna,40.743408,0.000000,73.519861,4.056431,2.524292,16.204513,-9.256592
3,1403302,56.0,boost PTV,boost/piers,57.551827,25.947701,100.000000,3.485419,2.439205,13.271245,1.551827
4,1402847,45.0,blizna PTV,blizna,44.069822,35.024182,95.508266,3.475949,2.429877,12.762535,-0.930178
5,1402419,45.0,blizna PTV,blizna,45.077445,40.803492,97.492848,4.747757,2.859463,15.976447,0.077445
6,1402186,56.0,boost PTV-plan,boost/piers,48.173865,39.911365,7.939509,4.980549,8.040788,2.804213,-7.826135
7,1401571,45.0,blizna PTV,blizna,45.882107,39.436020,96.552266,5.340130,14.190826,4.170746,0.882107
8,1400518,56.0,boost PTV-plan,boost/piers,56.301188,48.823600,95.033615,5.248780,9.586632,3.997892,0.301188
9,1399784,55.0,boost PTV-plan,boost/piers,54.468401,31.177600,85.547420,3.440787,3.149093,13.024975,-0.531599


In [162]:
df_18_batch_8_doucz.describe()

,TARGET_DOSE,PTV AVG,PTV D98%,PTV V95%,HEART AVG,LUNGS L AVG,LUNGS P AVG,PTV AVG - Target
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,51.000000,49.690276,35.874495,84.214006,4.333898,7.179143,8.444458,-1.309724
std,5.291503,5.725834,11.583399,22.588313,0.834513,4.686074,5.732264,3.156571
min,45.000000,40.743408,0.000000,7.939509,3.071679,2.422731,2.158029,-9.256592
25%,45.000000,44.573634,34.907850,82.180280,3.480684,2.575387,3.420737,-1.022762
50%,54.000000,48.173865,37.071451,89.982964,4.622347,8.040788,4.170746,-0.531599
75%,56.000000,55.081485,40.357429,96.030266,5.102798,11.479537,13.197605,0.559208
max,56.000000,57.551827,48.823600,100.000000,5.340130,14.190826,16.204513,1.551827


In [150]:
print("Prównanie trzech modeli")

df_c = pd.DataFrame()
df_c['PID'] = df50['PID']
df_c['PTV ROI'] = df50['PTV ROI']
df_c['TARGET_DOSE'] = df50['TARGET_DOSE']
df_c['PTV AVG Refrence PLAN'] = df50['PTV AVG']
df_c['PTV AVG 18_batch_8_doucz'] = df_18_batch_8_doucz['PTV AVG']
df_c['PTV AVG 50_e_31_41'] = df50_e_31_41['PTV AVG']
df_c['PTV AVG Refrence PLAN - TARGET'] = df50['PTV AVG'] - df50['TARGET_DOSE']
df_c['PTV AVG 18_batch_8_doucz - TARGET'] = df_18_batch_8_doucz['PTV AVG'] - df50['TARGET_DOSE']
df_c['PTV AVG 50_e_31_41 - TARGET'] = df50_e_31_41['PTV AVG'] - df50['TARGET_DOSE']
df_c


Prównanie trzech modeli


,PID,PTV ROI,TARGET_DOSE,PTV AVG Refrence PLAN,PTV AVG 18_batch_8_doucz,PTV AVG 50_e_31_41,PTV AVG Refrence PLAN - TARGET,PTV AVG 18_batch_8_doucz - TARGET,PTV AVG 50_e_31_41 - TARGET
0,1405662,blizna PTV,45.0,44.765133,43.911255,44.048645,-0.234867,-1.088745,-0.951355
1,1405045,boost PTV-plan,54.0,53.867287,54.817229,51.554879,-0.132713,0.817229,-2.445121
2,1404100,blizna PTV,50.0,39.427170,40.743408,40.042413,-10.572830,-9.256592,-9.957587
3,1403302,boost PTV,56.0,55.782528,57.551827,53.800313,-0.217472,1.551827,-2.199687
4,1402847,blizna PTV,45.0,44.537739,44.069822,43.152571,-0.462261,-0.930178,-1.847429
5,1402419,blizna PTV,45.0,44.733456,45.077445,44.650036,-0.266544,0.077445,-0.349964
6,1402186,boost PTV-plan,56.0,55.722382,48.173865,48.285828,-0.277618,-7.826135,-7.714172
7,1401571,blizna PTV,45.0,44.703739,45.882107,44.543987,-0.296261,0.882107,-0.456013
8,1400518,boost PTV-plan,56.0,55.796410,56.301188,55.148909,-0.203590,0.301188,-0.851091
9,1399784,boost PTV-plan,55.0,54.354881,54.468401,52.059168,-0.645119,-0.531599,-2.940832
